In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from aSPEM import aSPEM
import bayesianchangepoint as bcp

%matplotlib inline

In [2]:
from aSPEM import Analysis
e = Analysis(name_file_fit='fct_velocity_sigmo')

In [3]:
Full = e.Full_list(modes_bcp=[])

In [4]:
def mutual_information(p, data, bin_p=20, bin_data=30):
    
    x = p
    y = data

    support_x = np.linspace(min(x), max(x), bin_p)
    support_y = np.linspace(min(y), max(y), int(len(data)/bin_data))

    summation = 0.00
    for a in range(len(support_x)-1):
        for b in range(len(support_y)-1):
            ind_x = np.where((x>=support_x[a]) & (x<=support_x[a+1]))[0]
            ind_y = np.where((y>=support_y[b]) & (y<=support_y[b+1]))[0]

            px = len(ind_x) / len(x)
            py = len(ind_y) / len(x)
            pxy = len(np.where(np.in1d(ind_x, ind_y)==True)[0]) / len(x)

            if pxy>0.00: summation += pxy * np.log2(pxy / (px*py))


    return summation

In [5]:
modes_bcp = 'mean'
list_h = np.linspace(0, 0.1, 5)


for s in set(Full.sujet) :
    print('sujet =', s, end='  ')
    va = Full['va'][Full.sujet==s].values.tolist()
    bet = Full['results'][Full.sujet==s].values.tolist()
    
    mumu_va = []
    mumu_bet = []
    
    for h in list_h :
        x = np.array(Full['bino'][Full.sujet=='AM'])
        p_hat = np.zeros(len(x))
        for b in range(3):
            nb = 200*b
            liste = [0, 50, 100, 150, 200]
            for a in range(len(liste)-1) :
                p_bar, r_bar, beliefs = bcp.inference(x[nb+liste[a]:nb+liste[a+1]], h=h, p0=.5, r0=1.)
                p_hat_p, r_hat = bcp.readout(p_bar, r_bar, beliefs, mode=modes_bcp, p0=.5, fixed_window_size=40)
                p_hat[nb+liste[a]:nb+liste[a+1]] = p_hat_p

        
        proba = np.ma.masked_array(p_hat.tolist(), mask=np.isnan(va)).compressed()
        data = np.ma.masked_array(va, mask=np.isnan(va)).compressed()
        
        mumu_va.append(mutual_information(proba, data, bin_p=20, bin_data=30))
        
        
        
        proba = np.ma.masked_array(p_hat.tolist(), mask=np.isnan(bet)).compressed()
        data = np.ma.masked_array(bet, mask=np.isnan(bet)).compressed()
        
        mumu_bet.append(mutual_information(proba, data, bin_p=20, bin_data=30))
    print('va - h=%0.3f, mi=%0.3f'%(list_h[np.argmax(mumu_va)], max(mumu_va)), end='   ')
    print('bet - h=%0.3f, mi=%0.3f'%(list_h[np.argmax(mumu_bet)], max(mumu_bet)))

sujet = AM  va - h=0.050, mi=0.577   bet - h=0.025, mi=1.602
sujet = BMC  va - h=0.075, mi=0.730   bet - h=0.025, mi=1.739
sujet = DC  va - h=0.050, mi=0.749   bet - h=0.025, mi=1.281
sujet = OP  va - h=0.025, mi=0.526   bet - h=0.050, mi=1.227
sujet = LB  va - h=0.100, mi=0.880   bet - h=0.075, mi=1.481
sujet = IP  va - h=0.100, mi=0.730   bet - h=0.025, mi=1.906
sujet = RS  va - h=0.050, mi=0.645   bet - h=0.100, mi=0.717
sujet = FM  va - h=0.100, mi=0.838   bet - h=0.050, mi=1.229
sujet = YK  va - h=0.075, mi=0.508   bet - h=0.025, mi=0.621
sujet = CS  va - h=0.050, mi=0.792   bet - h=0.075, mi=1.133
sujet = SR  va - h=0.100, mi=1.033   bet - h=0.025, mi=1.574
sujet = TN  va - h=0.100, mi=0.932   bet - h=0.025, mi=1.372
